In [1]:
import subprocess as sp
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import sys, os
import signal
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb
from matplotlib import colors
from IPython.display import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('png')
plt.figure(figsize=(5,7))
import ipywidgets as widgets

<Figure size 360x504 with 0 Axes>

## Running VMD
### Collect inputs for running  VMD

This section first collect inputs to run VMD job that calculates the contact data. 
<br>1. Working directory: Directory that contains toppar and trajectory (*.dcd) files. Output will also be written in this directory.
<br>2. Name of toppar and dcd files (with extension)
<br>3. Two selection between which the contacts will get calculated. If the selctions are same, all intra-selection contacts will be calculated.
<br>4. Cutoff in Angstrom for finding contacts
<br>5. Location of Required tcl scripts: vmd_contact.tcl, bigdcd.tcl [these files will be copied to the working directory]
<br>6. Full path to the vmd executable

In [125]:
class data_input():
    def __init__(self, 
                 pwd = "/../../../", 
                 top = "*.prmtop or *.psf", 
                 dcd = "*.dcd",
                 sele1 = "all and noh",
                 sele2 = "all and noh",
                 cutoff = "4.5",
                 tcl_loc = "path to tcl files (bigdcd.tcl and vmd_contact.tcl)",
                 vmd_path = "full path to vmd executable"
                ):
        layout = widgets.Layout(width='auto', height='40px') #set width and height
        self.pwd = widgets.Text(description = 'Working Dir:',value = pwd, layout = layout)
        self.top = widgets.Text(description = 'Toppar file:',value = top, layout = layout)
        self.dcd = widgets.Text(description = 'Traj:',value = dcd, layout = layout) 
        self.sele1 = widgets.Text(description = 'selection-1:',value = sele1, layout = layout) 
        self.sele2 = widgets.Text(description = 'selection-2:',value = sele2, layout = layout)
        self.cutoff = widgets.Text(description = 'Cutoff (A):',value = cutoff, layout = layout)
        self.tcl_loc = widgets.Text(description = 'Tcl path:',value = tcl_loc, layout = layout)
        self.vmd_path = widgets.Text(description = 'VMD path:',value = vmd_path, layout = layout)
        self.pwd.on_submit(self.handle_submit)
        self.top.on_submit(self.handle_submit)
        self.dcd.on_submit(self.handle_submit)
        self.sele1.on_submit(self.handle_submit)
        self.sele2.on_submit(self.handle_submit)
        self.cutoff.on_submit(self.handle_submit)
        self.tcl_loc.on_submit(self.handle_submit)
        self.vmd_path.on_submit(self.handle_submit)
        display(self.pwd, self.top, self.dcd, self.sele1, self.sele2, self.cutoff, self.tcl_loc, self.vmd_path)

    def handle_submit(self, text):
        self.v = text.value
        return self.v

print("After input, press return in any field")
f = data_input()

After input, press return in any field


Text(value='/../../../', description='Working Dir:', layout=Layout(height='40px', width='auto'))

Text(value='*.prmtop or *.psf', description='Toppar file:', layout=Layout(height='40px', width='auto'))

Text(value='*.dcd', description='Traj:', layout=Layout(height='40px', width='auto'))

Text(value='all and noh', description='selection-1:', layout=Layout(height='40px', width='auto'))

Text(value='all and noh', description='selection-2:', layout=Layout(height='40px', width='auto'))

Text(value='4.5', description='Cutoff (A):', layout=Layout(height='40px', width='auto'))

Text(value='path to tcl files (bigdcd.tcl and vmd_contact.tcl)', description='Tcl path:', layout=Layout(height…

Text(value='full path to vmd executable', description='VMD path:', layout=Layout(height='40px', width='auto'))

#### Set path for the tcl files and copy to the working directory . . .

In [126]:
tcl_files = os.path.join(f.tcl_loc.value, "*.tcl")
# print(tcl_files)
copy = 'cp '+tcl_files+" "+f.pwd.value+"."
# print(copy)
sp.run(copy, shell=True)

/Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA-target/run/*.tcl


#### Check values of the given inputs . . .
If found discrepancy, check cell-2

In [128]:
print("Working Dir:", f.pwd.value)
print("toppar file:", f.top.value)
print("DCD file:", f.dcd.value)
print("Selection 1:", f.sele1.value)
print("Selection 2:", f.sele2.value)
print("Cutoff distance (Angstrom):", f.cutoff.value)
print("List of Content in the working directory", os.listdir(f.pwd.value))

Working Dir: /Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA-target/run/
toppar file: SYS_nowat.prmtop
DCD file: tgRNA_cl0-2.dcd
Selection 1: (resid  1 to 1153) and noh
Selection 2: (resid  1212 to 1241) and noh
Cutoff distance (Angstrom): 4.5
List of Content in the working directory ['run_vmd.sh', 'crRNA-atgRNA.dat', 'err.log', 'Contact-out_test.log', 'temp_contacts.dat', 'R1-R3_cl.c3.1265.pdb', 'crRNA-atgRNA_uniqueContact.npy', 'Cas13-crRNA_tgRNA_contact_cl0-2.npy', 'R1-R3_cl.c0.7068.pdb', 'compare_new.py', 'crRNA-tgRNA_uniqueContact.dat', 'temp.tcl', 'R1-R3_cl.c1', 'contact_book.html', 'Test-out1.log', 'R1-R3_cl.c1.2301.pdb', 'crRNA-tgRNA.npy', 'R1-R3_cl.c0', 'R1-R3_cl.c1.7633.pdb', 'Cas13-target_tgRNA_contact_cl0-2.dat', 'test.log', 'tgRNA_cl0-2-onlyProt.dcd', 'R1-R3_cl.c2.13334.pdb', 'Test-err.log', 'atgRNA-tgRNA.dat', 'tgRNA_cl0-2_cmap.npy', 'contact_final.tcl', 'contact_map.py', 'crRNA-tgRNA-5us-cl0-2.png', 'test.tcl', 'atgRNA-tgRNA_uniqueContact.np

#### Change to working directory . . .
Create the contact_final.tcl with the parsed inputs for calculation of contacts..

In [129]:
print("Current directory:", os.getcwd())
if os.getcwd()+"/" != f.pwd.value:
    os.chdir(f.pwd.value)
    print("\nNow in working directory:", os.getcwd())
else:
    print("\nAlready in working directory ....")
sed = 'sed '+'\"s/top_file/'+f.top.value+'/g; s/dcd_file/'+f.dcd.value+'/g; s/cutoff_dist/'+f.cutoff.value+'/g; s/selection1/'+f.sele1.value+'/g; s/selection2/'+f.sele2.value+'/g\" '+'vmd_contact.tcl >'+' contact_final.tcl'
print(sed)
sp.run(sed, shell=True)

Current directory: /Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA-target/run

Already in working directory ....
sed "s/top_file/SYS_nowat.prmtop/g; s/dcd_file/tgRNA_cl0-2.dcd/g; s/cutoff_dist/4.5/g; s/selection1/(resid  1 to 1153) and noh/g; s/selection2/(resid  1212 to 1241) and noh/g" vmd_contact.tcl > contact_final.tcl


CompletedProcess(args='sed "s/top_file/SYS_nowat.prmtop/g; s/dcd_file/tgRNA_cl0-2.dcd/g; s/cutoff_dist/4.5/g; s/selection1/(resid  1 to 1153) and noh/g; s/selection2/(resid  1212 to 1241) and noh/g" vmd_contact.tcl > contact_final.tcl', returncode=0)

#### Check for contact_all.dat (output of the VMD run) from any previous run . . . 
Delete if exists, otherwise output of the current run will get appended in the previous file..

In [130]:
if os.path.exists("contact_all.dat"):
  copy = 'cp '+"contact_all.dat"+" "+"contact_all.bak.dat"
  sp.run(copy, shell=True)
  os.remove("contact_all.dat")
  print("Backup created and deleted contact_all.dat file from previous run ....\n")
else:
  print("The file does not exist\n")

print("List of Content in the working directory", os.listdir(f.pwd.value))

Backup created and deleted contact_all.dat file from previous run ....

List of Content in the working directory ['run_vmd.sh', 'crRNA-atgRNA.dat', 'err.log', 'Contact-out_test.log', 'temp_contacts.dat', 'R1-R3_cl.c3.1265.pdb', 'crRNA-atgRNA_uniqueContact.npy', 'Cas13-crRNA_tgRNA_contact_cl0-2.npy', 'R1-R3_cl.c0.7068.pdb', 'compare_new.py', 'crRNA-tgRNA_uniqueContact.dat', 'temp.tcl', 'R1-R3_cl.c1', 'contact_book.html', 'Test-out1.log', 'R1-R3_cl.c1.2301.pdb', 'crRNA-tgRNA.npy', 'R1-R3_cl.c0', 'R1-R3_cl.c1.7633.pdb', 'Cas13-target_tgRNA_contact_cl0-2.dat', 'test.log', 'tgRNA_cl0-2-onlyProt.dcd', 'R1-R3_cl.c2.13334.pdb', 'Test-err.log', 'atgRNA-tgRNA.dat', 'tgRNA_cl0-2_cmap.npy', 'contact_final.tcl', 'contact_map.py', 'crRNA-tgRNA-5us-cl0-2.png', 'test.tcl', 'atgRNA-tgRNA_uniqueContact.npy', 'Contact-err_test.log', 'Cas13-target_tgRNA_contact_cl0-2.png', '.R1-R3_cl.c0.7068.pdb.swp', 'Contact-out.log', 'bigdcd.tcl', 'Test.vmd', 'crRNA-tgRNA_uniqueContact.npy', 'contact_all.bak.dat', 'Cas

### Run VMD
Output: 
1. contact_all.dat: 
<br> 1 line per contact per frame 
<br> Fields: frame number, contacting residues, minimal distance
<br> example:
<br> Say, if in frame 1, residue 361 (THR) is in contact with residue 363 (ASP) with minimal distance ~3.2 A
<br> 1       361 THR -   363 ASP - 3.197465
2. Contact-out.log & contact-err.log for stdout & stderr respectively
<br>
3. During the run, update of contact_all.dat can be checked by running the following command in the shell: "tail -f contact_all.dat"

In [131]:
outlogFile = open("Contact-out.log", "w")
errlogFile = open("Contact-err.log", "w")
p = sp.Popen([f.vmd_path.value, '-dispdev', 'none', '-e', 'contact_final.tcl'], stdout=outlogFile, stdin=sp.PIPE, stderr=errlogFile)
p.wait()
if p.poll is None:
    print("VMD still running..")
else:
    print("Run finished.. check whether the output contact_all.dat is okay")

Run finished.. check whether the output contact_all.dat is okay


## Calculate Contact Map
### Collect inputs
1. Working directoy: Directory that contains contact_all.dat (from VMD run). This directory will also be used as output directory.
<br>
2. Prefix of the output contact map (i.e. without extension)

In [2]:
class req_inputs():
    def __init__(self, 
                 pwd1 = "/../../../", 
                 out = "prefix of output map",
                 res_range1 = "provide the range of residues from selection1 [e.g. 1 to 1153]",
                 res_range2 = "provide the range of residues from selection2 [e.g. 1154 to 1204]"
                ):
        layout = widgets.Layout(width='auto', height='40px') #set width and height
        self.pwd1 = widgets.Text(description = 'Working Dir:',value = pwd1, layout = layout)
        self.out = widgets.Text(description = 'Outfile prefix:',value = out, layout = layout)
        self.res_range1 = widgets.Text(description = 'Res_range1:',value = res_range1, layout = layout)
        self.res_range2 = widgets.Text(description = 'Res_range2:',value = res_range2, layout = layout)
        self.pwd1.on_submit(self.handle_submit)
        self.out.on_submit(self.handle_submit)
        self.res_range1.on_submit(self.handle_submit)
        self.res_range2.on_submit(self.handle_submit)
        display(self.pwd1, self.out, self.res_range1, self.res_range2)

    def handle_submit(self, text):
        self.v = text.value
        return self.v

print("After input, press return in any field")
g = req_inputs()

After input, press return in any field


Text(value='/../../../', description='Working Dir:', layout=Layout(height='40px', width='auto'))

Text(value='prefix of output map', description='Outfile prefix:', layout=Layout(height='40px', width='auto'))

Text(value='provide the range of residues from selection1 [e.g. 1 to 1153]', description='Res_range1:', layout…

Text(value='provide the range of residues from selection2 [e.g. 1154 to 1204]', description='Res_range2:', lay…

### Set contact_all.dat and temp_contacts.dat (a temporary file for calculation of contact) . . .

In [132]:
contact = os.path.join(g.pwd1.value, "contact_all.dat")
temp_contact = os.path.join(g.pwd1.value, "temp_contacts.dat")
print(temp_contact)

/Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA-target/run/temp_contacts.dat


### Sorting contact map and get total residues, total frames . . .

In [133]:
res_range1 = len(np.arange(int(g.res_range1.value.split()[0]), int(g.res_range1.value.split()[2])+1))
res_range2 = len(np.arange(int(g.res_range2.value.split()[0]), int(g.res_range2.value.split()[2])+1))
print("Selection-1 length:",res_range1,"\n", "Selection-2 length:", res_range2, "\n")
s2 = 'sort -n -k1 '+contact+' | awk \'{print $1, $2, $5}\' > '+temp_contact
s4 = 'tail -n1 '+temp_contact+' | awk \'{print $1}\''
# print(s4) ## Checking the command
######### Output processing and system info #######
sp.run(s2, shell=True) # sort the output of vmd according to frames

frames=int(sp.getoutput(s4)) # Get total frames

print('No of frames:',frames)

1153 30


### Calculate the contact matrix . . . 

In [136]:
f2=1

#contactMatAll = np.zeros(shape=(frame_arr,res_arr,res_arr)) ## memory allocation issue with large trajs
running_contact =  np.zeros(shape=(res_range1,res_range2))
t =  np.zeros(shape=(res_range1,res_range2))

with open(temp_contact, "r") as input:
        for line in input:
                lines=line.split()
                f=int(lines[0])
                res1=int(lines[1]) - (int(g.res_range1.value.split()[0])
                res2=int(lines[2]) - (int(g.res_range2.value.split()[0])
                f1 = f
                if (f1 == f2):  ## to avoid repeating same contacts from same frames
                    running_contact[res1,res2] = 1
                    if g.res_range1.value == g.res_range2.value:
                        running_contact[res2,res1] = 1
                else:
                    t = np.add(t, running_contact) # updating after each frame
                    running_contact =  np.zeros(shape=(res_range1,res_range2))
                    running_contact[res1,res2] = 1
                    if g.res_range1.value == g.res_range2.value:
                        running_contact[res2,res1] = 1
                f2 = f1
                
input.close()
t = np.add(t, running_contact)

contactMat = np.true_divide(t, frames) # getting the fraction


if g.res_range1.value == g.res_range2.value:
    for x in range(res_range1):
        contactMat[x, x] = 1 ## same residue contact
        for y in range(x, res_range1):
           contactMat[x, y] = contactMat[y, x] ## for the lower diagonal

        
print("Saving the the map in a tabular format...\n")
print("Original residue ID's are gonna maintained as provided in the selction....\n")
#### --- we need a tabular data form -- ####
fo2 = open(os.path.join(g.pwd1.value, g.out.value)+".dat", "w+")
for i in range(res_range1):
    for j in range(res_range2):
        if (contactMat[i,j] != 0):
            fo2.write("%s %s %8.3f\n" % (i+int(g.res_range1.value.split()[0]), j+(int(g.res_range2.value.split()[0]), contactMat[i,j]))
            
fo2.close()
print("Done Saving.. check the *.dat file\n")

print("Saving the the map in *.npy format...\n")
print("It's just the matrix.. so only indexes for the provided range starts from 0\n")
print("Be careful if you are plotting the matrix directly.. you have to chenge the indexes accordingly..\n")

np.save(os.path.join(g.pwd1.value, g.out.value)+".npy", contactMat)
print("Done Saving.. \n")
print("Printing the contact map ...")
print(contactMat) ## check cmap

Saving the the map in a tabular format...

Original residue ID's are gonna maintained as provided in the selction....

Done Saving.. check the *.dat file

Saving the the map in *.npy format...

It's just the matrix.. so only indexes for the provided range starts from 0

Be careful if you are plotting the matrix directly.. you have to chenge the indexes accordingly..

Done Saving.. 

Printing the contact map ...
[[0.         0.         0.         ... 0.0674852  0.73824451 0.66875653]
 [0.         0.         0.         ... 0.55546848 0.91649251 0.54623824]
 [0.         0.         0.         ... 0.         0.01280042 0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


### Read contact data and plot . . . 

In [18]:
contact = np.loadtxt(os.path.join(g.pwd1.value, g.out.value)+".dat")
df_contact = pd.DataFrame(contact, columns = ['X','Y','Z'])

In [19]:
import matplotlib.gridspec as gridspec
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # high resolution

A = g.pwd1.value+g.out.value+".png"

@widgets.interact(min=(0, 1, 0.1), max=(0, 1, 0.1), save=(0, 1), figsize_x=(5,15), figsize_y=(5,15))
def update(min=0.3, max=0.9, save=0, size_x=8, size_y=8):
    df_contact_A = df_contact[((df_contact['Z']>min) & (df_contact['Z']<max))][['X','Y','Z']]
    fig = plt.figure(figsize=(figsize_x,figsize_y))
    fig = plt.figure()
    spec = gridspec.GridSpec(ncols=1, nrows=1, figure=fig)
    ax1 = fig.add_subplot(spec[0, 0])

    data = [df_contact_A]
    axss = [ax1]
    header = ['Contact Map']
    for l, n, i in zip(data, axss, header):
        X = n.scatter(l.X, l.Y, s=2, c=l.Z, cmap='coolwarm')
        n.axis([df_contact["X"].min(),df_contact["X"].max() + 10, df_contact["Y"].min(), df_contact["Y"].max() + 10])
        n.set_title(i, fontsize=16, fontweight='bold')
        n.set_xlabel('Selection 1 resids ->', fontsize=14, fontweight='bold') 
        n.set_ylabel('Selection 2 resids ->', fontsize=14, fontweight='bold')
        
        ## for setting up x-, y-ticks and ticklabels
        if (df_contact["X"].max() - df_contact["X"].min() > 500):
            n.set_xticks(np.arange(df_contact["X"].min(),df_contact["X"].max()+10,100))
            n.set_xticklabels(np.arange(df_contact["X"].min(),df_contact["X"].max()+10,100), fontsize=12)
        elif (df_contact["X"].max() - df_contact["X"].min() < 50):
            n.set_xticks(np.arange(df_contact["X"].min(),df_contact["X"].max()+10,5))
            n.set_xticklabels(np.arange(df_contact["X"].min(),df_contact["X"].max()+10,5), fontsize=12)
        else:
            n.set_xticks(np.arange(df_contact["X"].min(),df_contact["X"].max()+10,10))
            n.set_xticklabels(np.arange(df_contact["X"].min(),df_contact["X"].max()+10,10), fontsize=12)
        
        if (df_contact["Y"].max() - df_contact["Y"].min() > 500):
            n.set_xticks(np.arange(df_contact["Y"].min(),df_contact["Y"].max()+10,100))
            n.set_xticklabels(np.arange(df_contact["Y"].min(),df_contact["Y"].max()+10,100), fontsize=12)
        elif (df_contact["Y"].max() - df_contact["Y"].min() < 50):
            n.set_xticks(np.arange(df_contact["Y"].min(),df_contact["Y"].max()+10,5))
            n.set_xticklabels(np.arange(df_contact["Y"].min(),df_contact["Y"].max()+10,5), fontsize=12)
        else:
            n.set_xticks(np.arange(df_contact["Y"].min(),df_contact["Y"].max()+10,10))
            n.set_xticklabels(np.arange(df_contact["Y"].min(),df_contact["Y"].max()+10,10), fontsize=12)
            
        plt.colorbar(X,ax=n)
    plt.show()
    if save == 1:
        fig.savefig(A, dpi=300)


interactive(children=(FloatSlider(value=0.3, description='min', max=1.0), FloatSlider(value=0.9, description='…

<br>

## For comparison of contact maps
For Calculation of differential and unique contact maps by comparing 2 contact maps of same array size (i.e. maps calculated with same number of residues)
### Collect inputs
1. Working directoy: This directory will also be used as output directory.
<br>
2. Path to the first and second contact map (*.npy files)
<br>
3. Prefix of output maps (without extension)

In [3]:
class req_inputs2():
    def __init__(self,  
                 pwd2 = "/../../../",
                 CMap1_path = "/../../cmap1.npy (full path to the cmap1.npy)",
                 CMap2_path = "/../../cmap2.npy (full path to the cmap1.npy)",
                 out_pref = "crRNA-tgRNA",
                 res_range1 = "provide the range of residues from selection1 [e.g. 1 to 1153]",
                 res_range2 = "provide the range of residues from selection2 [e.g. 1154 to 1204]"
                ):
        layout = widgets.Layout(width='auto', height='40px') #set width and height
        self.pwd2 = widgets.Text(description = 'Working Dir:',value = pwd2, layout = layout)
        self.CMap1_path = widgets.Text(description = 'CMap1\n full path:',value = CMap1_path, layout = layout)
        self.CMap2_path = widgets.Text(description = 'CMap2 full path:',value = CMap2_path, layout = layout)
        self.out_pref = widgets.Text(description = 'Outfile prefix:',value = out_pref, layout = layout)
        self.pwd2.on_submit(self.handle_submit)
        self.CMap1_path.on_submit(self.handle_submit)
        self.CMap2_path.on_submit(self.handle_submit)
        self.out_pref.on_submit(self.handle_submit)
        self.res_range1.on_submit(self.handle_submit)
        self.res_range2.on_submit(self.handle_submit)
        display(self.pwd2,self.CMap1_path, self.CMap2_path, self.out_pref, self.res_range1, self.res_range2)

    def handle_submit(self, text):
        self.v = text.value
        return self.v

print("After input, press return in any field")
h = req_inputs2()

After input, press return in any field


Text(value='/../../../', description='Working Dir:', layout=Layout(height='40px', width='auto'))

Text(value='/../../cmap1.npy (full path to the cmap1.npy)', description='CMap1\n full path:', layout=Layout(he…

Text(value='/../../cmap2.npy (full path to the cmap1.npy)', description='CMap2 full path:', layout=Layout(heig…

Text(value='crRNA-tgRNA', description='Outfile prefix:', layout=Layout(height='40px', width='auto'))

#### Check values of the given inputs . . .
If found discrepancy, check input cell

In [7]:
print("Working Dir:", h.pwd2.value)
print("CMap1:", h.CMap1_path.value)
print("CMap2:", h.CMap2_path)
print("Out Prefix:", h.out_pref.value)
print("Selection 1:", h.res_range1.value)
print("Selection 2:", h.res_range2.value)

Working Dir: /Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA-target/run/
CMap1: /Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA/run/crRNA_cl0-2_cmap.npy
CMap2: Text(value='/Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-Antitag/run/atgRNA_cl0-2_cmap.npy', description='CMap2 full path:')
Out Prefix: crRNA-atgRNA


### Calculation of differential and unique contact map
"Differential" contacts are common in both the input maps, but differs in their stability. The output differential contact map will give the difference in the stability in a scale of -1 to +1. Positive (+ve) values correspond to stability in the first input and negative (-ve) values correspond to stability in the second input contact map.
<br>
<br>
"Unique" contacts are exclusive in either of the system. Stability scale is similar to differential map. Positive (+ve) values correspond to contact in the first input and negative (-ve) values correspond to the second input contact map.

<br>
Output: Output contact matrices will be written as *.npy and as *.dat file (tabular data form)

In [9]:
a1 = np.load(h.CMap1_path.value)
a2 = np.load(h.CMap2_path.value)

ncols1 = a1.shape[0]
ncols2 = a1.shape[1]

t1 =  np.zeros(shape=(ncols1,ncols2))
t2 =  np.zeros(shape=(ncols1,ncols2))

# for i in range(ncols):
#     for j in range(i, ncols):
for i in range(ncols1):
    for j in range(ncols2):
        if ((a1[i,j] != 0) & (a2[i,j] != 0)):
            t1[i,j] = a1[i,j] - a2[i,j] ## differential contact map #######
        else:
            t2[i,j] = a1[i,j] - a2[i,j] ## unique contact map #######


np.save(os.path.join(h.pwd2.value, h.out_pref.value)+"_diff.npy", t1)
np.save(os.path.join(h.pwd2.value, h.out_pref.value)+"_uniq.npy", t2)

#### --- we need a tabular data form -- ####
fo2 = open(os.path.join(h.pwd2.value, h.out_pref.value)+"_diff.dat", "w+")
fo3 = open(os.path.join(h.pwd2.value, h.out_pref.value)+"_uniq.dat", "w+")


if ncols1 = ncols2:
    print(" We have got symmentric map (same number of residues in rows and columns)..")
    print( "With a symmetric map, we are gonna output the tabular data in a way that plotting would show stable 1st input contacts in the upper diagona \
            and stable 2nd input contact in the lower diagonal...)
    for i in range(ncols1):
        for j in range(ncols2):
            if (t1[i,j] != 0):
                if (t1[i,j] > 0): ## 1st system stable
                     fo2.write("%s %s %8.3f\n" % (i+int(h.res_range1.value.split()[0]), j+(int(h.res_range2.value.split()[0]), t1[i,j])))
                else:    ## 2nd system stable
                     fo2.write("%s %s %8.3f\n" % (j+int(h.res_range2.value.split()[0]), i+(int(h.res_range1.value.split()[0]), t1[i,j])))

            if (t2[i,j] != 0):
                if (t2[i,j] > 0): ## 1st system stable
                    fo3.write("%s %s %8.3f\n" % (i+int(h.res_range1.value.split()[0]), j+(int(h.res_range2.value.split()[0]), t2[i,j]))
                else:
                    fo3.write("%s %s %8.3f\n" % (j+int(h.res_range2.value.split()[0]), i+(int(h.res_range1.value.split()[0]), t2[i,j]))
else:

    for i in range(ncols1):
        for j in range(ncols2):
            if (t1[i,j] != 0):
                 fo2.write("%s %s %8.3f\n" % (i+int(h.res_range1.value.split()[0]), j+(int(h.res_range2.value.split()[0]), t1[i,j]))

            if (t2[i,j] != 0):
                fo3.write("%s %s %8.3f\n" % (i+int(h.res_range1.value.split()[0]), j+(int(h.res_range2.value.split()[0]), t2[i,j]))

fo2.close()
fo3.close()

<br>

## Plotting of Differential and unique contact maps

In [2]:
class req_inputs3():
    def __init__(self,  
                 pwd3 = "/../../../",
                 dMap_path = "/../../differential_cmap.dat (path to the differential contact map)",
                 uMap_path = "/../../uniq_cmap.dat (path to the uniq contact map)",
                 out_pref = "crRNA-tgRNA-5us-cl0-2"
                ):
        layout = widgets.Layout(width='auto', height='40px') #set width and height
        self.pwd3 = widgets.Text(description = 'Working Dir:',value = pwd3, layout = layout)
        self.dMap_path = widgets.Text(description = 'Diff. map:',value = dMap_path, layout = layout)
        self.uMap_path = widgets.Text(description = 'Uniq. map:',value = uMap_path, layout = layout)
        self.out_pref = widgets.Text(description = 'Outfile prefix:',value = out_pref, layout = layout)
        self.pwd3.on_submit(self.handle_submit)
        self.dMap_path.on_submit(self.handle_submit)
        self.uMap_path.on_submit(self.handle_submit)
        self.out_pref.on_submit(self.handle_submit)
        display(self.pwd3,self.dMap_path, self.uMap_path, self.out_pref)

    def handle_submit(self, text):
        self.v = text.value
        return self.v

print("After input, press return in any field")
i = req_inputs3()

After input, press return in any field


Text(value='/../../../', description='Working Dir:', layout=Layout(height='40px', width='auto'))

Text(value='/../../differential_cmap.dat (path to the differential contact map)', description='Diff. map:', la…

Text(value='/../../uniq_cmap.dat (path to the uniq contact map)', description='Uniq. map:', layout=Layout(heig…

Text(value='crRNA-tgRNA-5us-cl0-2', description='Outfile prefix:', layout=Layout(height='40px', width='auto'))

In [3]:
Diff = np.loadtxt(i.dMap_path.value)
uniq = np.loadtxt(i.uMap_path.value)
df_diff = pd.DataFrame(Diff, columns = ['X','Y','Z'])
df_uniq = pd.DataFrame(uniq, columns = ['X','Y','Z'])

In [6]:
import matplotlib.gridspec as gridspec
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # high resolution

A = i.pwd3.value+i.out_pref.value+".png"

@widgets.interact(min1=(0, 1, 0.1), max1=(0, 1, 0.1), min2=(-1, 0, 0.1), max2=(-1, 0, 0.1), save=(0, 1), figsize_x=(5,15), figsize_y=(5,15))
def update(min1=0.3, max1=0.9, min2=-0.9, max2=-0.3, save=0, figsize_x=8, figsize_y=8):
    df_diff_A = df_diff[((df_diff['Z']>min1) & (df_diff['Z']<max1)) | ((df_diff['Z']< max2) & (df_diff['Z'] > min2)) ][['X','Y','Z']]
    df_uniq_A = df_uniq[((df_uniq['Z']>min1) & (df_uniq['Z']<max1)) | ((df_uniq['Z']< max2) & (df_uniq['Z'] > min2)) ][['X','Y','Z']]
    fig = plt.figure(figsize=(figsize_x,figsize_y))
    spec = gridspec.GridSpec(ncols=1, nrows=2, hspace=0.3, figure=fig)
    ax1 = fig.add_subplot(spec[0, 0])
    ax2 = fig.add_subplot(spec[1, 0])

    data = [df_diff_A, df_uniq_A]
    axss = [ax1, ax2]
    header = ['Differential Map', 'Unique Map']
    for l, n, i in zip(data, axss, header):
        X = n.scatter(l.X, l.Y, s=2, c=l.Z, cmap='coolwarm')
        n.axis([0,1153, 0, 1153])
        n.axis([df_diff["X"].min(),df_diff["X"].max() + 10, df_diff["Y"].min(), df_diff["Y"].max() + 10])
        # n.legend(frameon=False)
        n.set_title(i, fontsize=16, fontweight='bold')
        n.set_xlabel('Residues', fontsize=14, fontweight='bold') 
        n.set_ylabel('Residues', fontsize=14, fontweight='bold')
        
        ## for setting up x-, y-ticks and ticklabels
        if (df_diff["X"].max() - df_diff["X"].min() > 500):
            n.set_xticks(np.arange(df_diff["X"].min(),df_diff["X"].max()+10,100))
            n.set_xticklabels(np.arange(df_diff["X"].min(),df_diff["X"].max()+10,100), fontsize=12)
        elif (df_diff["X"].max() - df_diff["X"].min() < 50):
            n.set_xticks(np.arange(df_diff["X"].min(),df_diff["X"].max()+10,5))
            n.set_xticklabels(np.arange(df_diff["X"].min(),df_diff["X"].max()+10,5), fontsize=12)
        else:
            n.set_xticks(np.arange(df_diff["X"].min(),df_diff["X"].max()+10,10))
            n.set_xticklabels(np.arange(df_diff["X"].min(),df_diff["X"].max()+10,10), fontsize=12)
        
        if (df_diff["Y"].max() - df_diff["Y"].min() > 500):
            n.set_xticks(np.arange(df_diff["Y"].min(),df_diff["Y"].max()+10,100))
            n.set_xticklabels(np.arange(df_diff["Y"].min(),df_diff["Y"].max()+10,100), fontsize=12)
        elif (df_diff["Y"].max() - df_diff["Y"].min() < 50):
            n.set_xticks(np.arange(df_diff["Y"].min(),df_diff["Y"].max()+10,5))
            n.set_xticklabels(np.arange(df_diff["Y"].min(),df_diff["Y"].max()+10,5), fontsize=12)
        else:
            n.set_xticks(np.arange(df_diff["Y"].min(),df_diff["Y"].max()+10,10))
            n.set_xticklabels(np.arange(df_diff["Y"].min(),df_diff["Y"].max()+10,10), fontsize=12)
        
        plt.colorbar(X,ax=n)
    plt.show()
    if save == 1:
        fig.savefig(A, dpi=300)


interactive(children=(FloatSlider(value=0.3, description='min1', max=1.0), FloatSlider(value=0.9, description=…